# Forecasting pipeline (Colab-ready)
# This notebook generates a synthetic ecommerce demand dataset and runs ARIMA and LSTM forecasting.
# Author: Generated by script for Colab


In [ ]:
%%bash
# Install required packages (Colab-friendly). Note: reinstalling tensorflow may restart runtime.
python -V
pip -q install scikit-learn==1.3.2 tensorflow==2.13.0 statsmodels==0.14.6 pandas==2.3.3 matplotlib==3.10.8 numpy==2.3.5 nbformat -q
python -c "import sklearn, tensorflow as tf, statsmodels, pandas, matplotlib, numpy; print('sklearn', sklearn.__version__); print('tensorflow', tf.__version__); print('statsmodels', statsmodels.__version__); print('pandas', pandas.__version__); print('matplotlib', matplotlib.__version__); print('numpy', numpy.__version__)"
# Show if GPU available
python -c "import tensorflow as tf; print('GPU available:', tf.config.list_physical_devices('GPU'))"


In [ ]:
# Deterministic seeding for reproducibility
import os
import random
import numpy as np
import tensorflow as tf

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
try:
    tf.random.set_seed(SEED)
    if hasattr(tf.config.experimental, 'enable_op_determinism'):
        tf.config.experimental.enable_op_determinism()
except Exception:
    pass
print('Seed set to', SEED)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# Imports used in the notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print('Imports OK')


In [ ]:
# Data generation (ported from generate_ecommerce_demand.py, compacted for notebook)
import numpy as np
import pandas as pd

START_DATE = '2022-01-01'
END_DATE = '2024-12-31'
SKUS = ['SKU_001','SKU_002','SKU_003']

RANDOM_SEED=SEED
np.random.seed(RANDOM_SEED)

dates = pd.date_range(start=START_DATE,end=END_DATE,freq='D')
n_days = len(dates)
day_idx = np.arange(n_days)
weekday = dates.weekday.values
day_of_year = dates.dayofyear.values

# Parameters
BASE_DEMAND = {'SKU_001':80.0,'SKU_002':45.0,'SKU_003':18.0}
BASE_PRICE = {'SKU_001':19.99,'SKU_002':9.99,'SKU_003':49.99}
WEEKLY_STRENGTH = {'SKU_001':0.20,'SKU_002':0.28,'SKU_003':0.12}
YEARLY_STRENGTH = {'SKU_001':0.30,'SKU_002':0.22,'SKU_003':0.08}
TREND_FRACTION = {'SKU_001':0.12,'SKU_002':0.10,'SKU_003':0.18}

rows = []
for sku in SKUS:
    base = BASE_DEMAND[sku]
    base_price = BASE_PRICE[sku]
    weekly_component = WEEKLY_STRENGTH[sku] * base * np.sin(2 * np.pi * day_idx / 7)
    yearly_component = YEARLY_STRENGTH[sku] * base * np.sin(2 * np.pi * day_of_year / 365.25)
    trend_total = base * TREND_FRACTION[sku]
    trend = trend_total * (day_idx / float(n_days - 1))
    noise_std = max(1.0, base * 0.12)
    noise = np.random.normal(loc=0.0, scale=noise_std, size=n_days)

    promo_p = np.random.uniform(0.15,0.25)
    if sku == 'SKU_003':
        promotion_flag = np.zeros(n_days, dtype=int)
        expected_promo_days = int(np.round(promo_p * n_days))
        avg_len = 4
        num_windows = max(1, expected_promo_days // max(1, avg_len))
        starts = np.random.choice(np.arange(n_days), size=num_windows, replace=False)
        for s in starts:
            length = np.random.randint(2,8)
            end = min(n_days, s + length)
            promotion_flag[s:end] = 1
        discount_pct = np.zeros(n_days)
        window_days = promotion_flag.astype(bool)
        if window_days.sum() > 0:
            discount_pct[window_days] = np.random.uniform(0.08,0.25,size=window_days.sum())
        promo_lift = np.random.uniform(0.25 * 1.2, 0.60 * 1.2)
    else:
        promotion_flag = np.random.binomial(1, promo_p, size=n_days)
        promo_lift = np.random.uniform(0.25, 0.60)
        will_discount = (np.random.rand(n_days) < 0.6) & (promotion_flag == 1)
        discount_pct = np.zeros(n_days)
        if will_discount.sum() > 0:
            discount_pct[will_discount] = np.random.uniform(0.05,0.20,size=will_discount.sum())

    # Price
    price = np.empty(n_days, dtype=float)
    price[0] = base_price * (1.0 + np.random.normal(0.0, 0.003))
    if sku == 'SKU_003':
        min_factor = 0.7
        max_factor = 1.3
        reversion_rate = 0.02
        local_price_noise = 0.007 * 3.0
    else:
        min_factor = 0.8
        max_factor = 1.2
        reversion_rate = 0.04
        local_price_noise = 0.007
    for t in range(1, n_days):
        drift = reversion_rate * (base_price - price[t - 1])
        p_noise = price[t - 1] * np.random.normal(0.0, local_price_noise)
        price[t] = price[t - 1] + drift + p_noise
        if discount_pct[t] > 0:
            price[t] = price[t] * (1.0 - discount_pct[t])
        price[t] = float(np.clip(price[t], base_price * min_factor, base_price * max_factor))
        price[t] += np.random.normal(0.0, 0.03)
    price = np.round(price.clip(min=0.01), 2)

    price_mean = price.mean()
    PRICE_ELASTICITY_SKU = {'SKU_001':-1.5,'SKU_002':-1.5,'SKU_003':-0.8}
    price_effect = PRICE_ELASTICITY_SKU[sku] * (price - price_mean) / price_mean * base

    units_raw = base + trend + weekly_component + yearly_component + noise + price_effect

    shocks = (np.random.rand(n_days) < 0.01)
    if shocks.any():
        shock_mults = np.random.uniform(1.5,3.0,size=shocks.sum())
        units_raw[shocks] = units_raw[shocks] * shock_mults

    units_promoted = units_raw * (1.0 + (promotion_flag * promo_lift))
    lam = np.clip(units_promoted, a_min=0.1, a_max=None)
    if sku == 'SKU_003':
        shape = 0.9
        lam_gamma = np.random.gamma(shape, (lam / shape).clip(min=1e-6))
        units_final = np.random.poisson(lam_gamma).astype(int)
    else:
        units_final = np.random.poisson(lam).astype(int)
    units_final = np.maximum(0, units_final)

    rows.append(pd.DataFrame({
        'date': dates.strftime('%Y-%m-%d'),
        'sku_id': [sku] * n_days,
        'units_sold': units_final,
        'price': price,
        'promotion_flag': promotion_flag
    }))

# Concatenate and save
df_gen = pd.concat(rows, ignore_index=True).sort_values(['date','sku_id']).reset_index(drop=True)
df_gen.to_csv('ecommerce_demand.csv', index=False)
print('Wrote ecommerce_demand.csv with shape', df_gen.shape)
print(df_gen.head(8).to_string(index=False))
print('\nPromotion fraction per SKU:')
print(df_gen.groupby('sku_id')['promotion_flag'].mean())


In [ ]:
# Load and aggregate to daily totals
df = pd.read_csv('ecommerce_demand.csv', parse_dates=['date'])
agg = df.groupby('date').agg(total_units=('units_sold','sum'), avg_price=('price','mean'), promo_any=('promotion_flag','max'))
full_idx = pd.date_range(start=agg.index.min(), end=agg.index.max(), freq='D')
agg = agg.reindex(full_idx)
agg.index.name = 'date'
agg['total_units'] = agg['total_units'].fillna(0).astype(int)
agg['avg_price'] = agg['avg_price'].fillna(method='ffill').fillna(method='bfill')
agg['promo_any'] = agg['promo_any'].fillna(0).astype(int)

TEST_DAYS = 90
train_end = agg.index[-(TEST_DAYS + 1)]
print('Data range:', agg.index.min().date(), 'to', agg.index.max().date())
print('Train ends at:', train_end.date(), 'Test days:', TEST_DAYS)
agg.head()


In [ ]:
# ARIMA: select order and forecast
from statsmodels.tsa.arima.model import ARIMA

def select_arima_order(series, p_max=3, d_vals=(0,1), q_max=3):
    best_aic = float('inf')
    best_order = (0,0,0)
    for d in d_vals:
        for p in range(p_max+1):
            for q in range(q_max+1):
                try:
                    res = ARIMA(series, order=(p,d,q)).fit(method='innovations_mle', disp=0)
                    if res.aic < best_aic:
                        best_aic = res.aic
                        best_order = (p,d,q)
                except Exception:
                    continue
    return best_order

series = agg['total_units']
train_series = series[:train_end]
order = select_arima_order(train_series)
print('Selected ARIMA order:', order)
model = ARIMA(train_series, order=order)
res = model.fit()
steps = TEST_DAYS
forecast = res.get_forecast(steps=steps).predicted_mean
forecast_index = pd.date_range(start=train_series.index[-1] + pd.Timedelta(days=1), periods=steps)
arima_forecast = pd.Series(forecast.values, index=forecast_index)

# Evaluate
actual = series.loc[arima_forecast.index]
ARIMA_RMSE = math.sqrt(mean_squared_error(actual.values, arima_forecast.values))
ARIMA_MAPE = (np.mean(np.abs((actual.values - arima_forecast.values) / (np.abs(actual.values) + 1e-8))) * 100)
print(f'ARIMA RMSE: {ARIMA_RMSE:.4f}, ARIMA MAPE: {ARIMA_MAPE:.2f}%')

# Plot ARIMA
plt.figure(figsize=(10,4))
plt.plot(actual.index, actual.values, label='Actual', color='black')
plt.plot(arima_forecast.index, arima_forecast.values, label='ARIMA forecast')
plt.title('ARIMA: Actual vs Forecast')
plt.legend()
plt.show()


In [ ]:
# LSTM: prepare sequences, train, and forecast recursively
from sklearn.preprocessing import MinMaxScaler

N_IN = 30
features = ['total_units','avg_price','promo_any']
arr = agg[features].values.astype(float)
train_mask = agg.index <= train_end
train_arr = arr[train_mask]
test_arr = arr[~train_mask]

scaler = MinMaxScaler()
scaler.fit(train_arr)
arr_scaled = scaler.transform(arr)

# create sliding windows
X, y = [], []
for i in range(N_IN, len(arr_scaled)):
    X.append(arr_scaled[i - N_IN:i, :])
    y.append(arr_scaled[i, 0])
X = np.array(X); y = np.array(y)
train_len = train_arr.shape[0]
train_count = max(0, train_len - N_IN)
X_train = X[:train_count]
y_train = y[:train_count]

print('LSTM training samples:', X_train.shape)

# build model
model = Sequential([LSTM(64, input_shape=(N_IN, len(features))), Dropout(0.2), Dense(32, activation='relu'), Dense(1)])
model.compile(optimizer='adam', loss='mse')
EPOCHS = 30
es = EarlyStopping(patience=8, restore_best_weights=True, monitor='loss')
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=32, callbacks=[es], verbose=1)

# recursive forecast on scaled space
steps = test_arr.shape[0]
last_window = arr_scaled[train_len - N_IN:train_len].copy()
preds_scaled = []
for s in range(steps):
    x = last_window.reshape((1, N_IN, len(features)))
    yhat_scaled = model.predict(x, verbose=0)[0,0]
    preds_scaled.append(yhat_scaled)
    exog_scaled = scaler.transform(test_arr[s].reshape(1,-1))[0]
    exog_scaled[0] = yhat_scaled
    last_window = np.vstack([last_window[1:], exog_scaled])

# inverse transform preds
preds_scaled_arr = np.array(preds_scaled).reshape(-1,1)
dummy = np.zeros((len(preds_scaled), arr.shape[1]))
dummy[:,0:1] = preds_scaled_arr
dummy[:,1:] = scaler.transform(test_arr)[:,1:]
inv = scaler.inverse_transform(dummy)
lstm_preds = inv[:,0]
forecast_index = agg.index[agg.index > train_end]
lstm_forecast = pd.Series(lstm_preds, index=forecast_index)

# Evaluate
LSTM_RMSE = math.sqrt(mean_squared_error(agg['total_units'].loc[forecast_index].values, lstm_forecast.values))
LSTM_MAPE = (np.mean(np.abs((agg['total_units'].loc[forecast_index].values - lstm_forecast.values) / (np.abs(agg['total_units'].loc[forecast_index].values) + 1e-8))) * 100)
print(f'LSTM RMSE: {LSTM_RMSE:.4f}, LSTM MAPE: {LSTM_MAPE:.2f}%')

plt.figure(figsize=(10,4))
plt.plot(agg['total_units'].loc[forecast_index].index, agg['total_units'].loc[forecast_index].values, label='Actual', color='black')
plt.plot(lstm_forecast.index, lstm_forecast.values, label='LSTM forecast')
plt.title('LSTM: Actual vs Forecast')
plt.legend()
plt.show()


In [ ]:
# Combine plots and save, and show metrics
plt.figure(figsize=(12,6))
plt.plot(agg['total_units'].loc[forecast_index].index, agg['total_units'].loc[forecast_index].values, label='Actual', color='black')
plt.plot(arima_forecast.index, arima_forecast.values, label='ARIMA Forecast')
plt.plot(lstm_forecast.index, lstm_forecast.values, label='LSTM Forecast')
plt.title('Actual vs Forecasted Daily Demand')
plt.xlabel('Date')
plt.ylabel('Total Units')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('actual_vs_predicted.png')
plt.show()

metrics = pd.DataFrame({
    'model': ['ARIMA','LSTM'],
    'rmse': [ARIMA_RMSE, LSTM_RMSE],
    'mape': [ARIMA_MAPE, LSTM_MAPE]
})
print(metrics)
metrics.to_csv('forecast_metrics.csv', index=False)
print('Saved actual_vs_predicted.png and forecast_metrics.csv')


In [ ]:
# Additional section: programmatically create another notebook and provide download link (per outline)
import nbformat
from nbformat.v4 import new_notebook, new_markdown_cell, new_code_cell

# Simple example notebook that demonstrates how to serialize and download
cells = [
    new_markdown_cell('# Generated Notebook\nThis file was programmatically generated from the main notebook.'),
    new_code_cell("print('Hello from generated notebook')")
]
nb = new_notebook(cells=cells, metadata={"language_info": {"name": "python"}})
nbfpath = 'generated_forecast.ipynb'
nbformat.write(nb, nbfpath)
print('Wrote', nbfpath)

# Validate by reading it back
nb2 = nbformat.read(nbfpath, as_version=4)
print('Generated notebook has cells:', len(nb2['cells']))

# Provide download link in Colab
try:
    from google.colab import files
    files.download(nbfpath)
except Exception:
    print('google.colab.files not available in this environment; download manually if needed')


## Final notes

- Default EPOCHS in LSTM set to 30 for interactive Colab speed; increase to 100 for better performance.
- Use GPU runtime in Colab for faster training (Runtime > Change runtime type > GPU).
- The notebook is deterministic given the seed, but exact floating-point reproducibility may vary with different TF builds.

Thank you — adjust parameters at the top of the notebook (SEED, EPOCHS, N_IN) to experiment.


In [ ]:
# Add time features used by improved LSTM
agg['dow'] = agg.index.weekday.astype(int)
agg['dow_sin'] = np.sin(2 * np.pi * agg['dow'] / 7)
agg['dow_cos'] = np.cos(2 * np.pi * agg['dow'] / 7)

day_of_year = agg.index.dayofyear.values
agg['doy_sin'] = np.sin(2 * np.pi * day_of_year / 365.25)
agg['doy_cos'] = np.cos(2 * np.pi * day_of_year / 365.25)

agg.head()


In [ ]:
# Improved LSTM: multi-step (7-day block) forecasting with validation and callbacks
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

N_IN = 30
N_OUT = 7
features = ['total_units','avg_price','promo_any','dow_sin','dow_cos','doy_sin','doy_cos']
arr = agg[features].values.astype(float)

train_mask = agg.index <= train_end
train_arr = arr[train_mask]
test_arr = arr[~train_mask]

scaler = MinMaxScaler(); scaler.fit(train_arr)
arr_scaled = scaler.transform(arr)

# create multi-step windows
X, Y = [], []
for i in range(N_IN, len(arr_scaled) - N_OUT + 1):
    X.append(arr_scaled[i - N_IN:i, :])
    Y.append(arr_scaled[i:i+N_OUT, 0])
X = np.array(X); Y = np.array(Y)

train_len = train_arr.shape[0]
train_count = max(0, train_len - N_IN - (N_OUT - 1) + 1)
X_train = X[:train_count]
Y_train = Y[:train_count]

# validation split
val_split = max(1, int(0.1 * X_train.shape[0]))
X_val = X_train[-val_split:]; Y_val = Y_train[-val_split:]
X_train = X_train[:-val_split]; Y_train = Y_train[:-val_split]

# build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
model = Sequential([LSTM(128, input_shape=(N_IN, len(features))), Dropout(0.2), Dense(64, activation='relu'), Dense(N_OUT)])
model.compile(optimizer='adam', loss='mse')

# callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True), ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7), ModelCheckpoint('colab_lstm_best.h5', monitor='val_loss', save_best_only=True)]

model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=100, batch_size=32, callbacks=callbacks, verbose=1)

# Forecast test horizon in blocks
steps = test_arr.shape[0]
last_window = arr_scaled[train_len - N_IN:train_len].copy()
idx = 0
preds_scaled = []
while idx < steps:
    x = last_window.reshape((1, N_IN, arr.shape[1]))
    yhat_block = model.predict(x, verbose=0)[0]
    block_len = min(N_OUT, steps - idx)
    exog_block = test_arr[idx:idx+block_len]
    exog_block_scaled = scaler.transform(exog_block)
    exog_block_scaled[:block_len, 0] = yhat_block[:block_len]
    preds_scaled.extend(yhat_block[:block_len].tolist())
    last_window = np.vstack([last_window[block_len:], exog_block_scaled])
    idx += block_len

preds_scaled = np.array(preds_scaled).reshape(-1,1)
dummy = np.zeros((len(preds_scaled), arr.shape[1]))
dummy[:,0:1] = preds_scaled
dummy[:,1:] = scaler.transform(test_arr)[:,1:]
inv = scaler.inverse_transform(dummy)
lstm_preds = inv[:,0]

forecast_index = agg.index[agg.index > train_end]

# Evaluate
from sklearn.metrics import mean_squared_error
rmse_lstm = math.sqrt(mean_squared_error(agg['total_units'].loc[forecast_index].values, lstm_preds))
mape_lstm = np.mean(np.abs((agg['total_units'].loc[forecast_index].values - lstm_preds) / (np.abs(agg['total_units'].loc[forecast_index].values) + 1e-8))) * 100
print('LSTM RMSE:', rmse_lstm, 'LSTM MAPE:', mape_lstm)

# quick plot
plt.figure(figsize=(12,4))
plt.plot(agg['total_units'].loc[forecast_index].index, agg['total_units'].loc[forecast_index].values, label='Actual', color='black')
plt.plot(forecast_index, lstm_preds, label='Improved LSTM (block)', alpha=0.9)
plt.legend(); plt.show()
